# Imports

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json

# Read the file

In [2]:
products = pd.read_csv("..\..\data\products.csv")

In [3]:
products.shape

(500000, 3)

In [4]:
products.head()

,name,sku,description
0,Bryce Jones,lay-raise-best-end,Art community floor adult your single type. Pe...
1,John Robinson,cup-return-guess,Produce successful hot tree past action young ...
2,Theresa Taylor,step-onto,Choice should lead budget task. Author best me...
3,Roger Huerta,citizen-some-middle,Important fight wrong position fine. Friend so...
4,John Buckley,term-important,Alone maybe education risk despite way.\nWant ...


In [30]:
products.loc[0, :] = ['Bryce Jobes', 'lay-raise-best-end', products.loc[0, 'description']]

In [31]:
products.head()

,name,sku,description
0,Bryce Jobes,lay-raise-best-end,Art community floor adult your single type. Pe...
1,John Robinson,cup-return-guess,Produce successful hot tree past action young ...
2,Theresa Taylor,step-onto,Choice should lead budget task. Author best me...
3,Roger Huerta,citizen-some-middle,Important fight wrong position fine. Friend so...
5,Tiffany Johnson,do-many-avoid,Born tree wind.\nBoy marriage begin value. Rec...


# Processing

Given that "sku" is not unique but is to be considered as primary key:
Assumption: We are following SCD 1 keeping the last record in file

In [5]:
products.drop_duplicates(subset=['sku'],keep='last',inplace=True)

In [6]:
products.shape

(466693, 3)

## Read Config

In [13]:
with open('../../config/config.json') as fp:
    config = json.load(fp)
# config

##  Create engine

In [14]:
engine=create_engine(config['connection_strings']['dummy'])

In [11]:
query = """select * from products where sku in %s""" % str(tuple(products.sku.tolist()))

In [18]:
from datetime import datetime

In [19]:
start = datetime.now()
existing_data = pd.read_sql(sql=query, con=engine)
end = datetime.now()
end - start

datetime.timedelta(seconds=107, microseconds=317346)

In [23]:
new_products = products.loc[~products['sku'].isin(existing_data['sku'])]

In [27]:
_procducts = products.loc[products['sku'].isin(existing_data['sku'])]

In [24]:
products.columns

Index(['name', 'sku', 'description'], dtype='object')

In [32]:
updated_products = products.loc[(products['name']!=_procducts['name'])|(products['description']!=_procducts['description'])]

In [33]:
updated_products

,name,sku,description
0,Bryce Jobes,lay-raise-best-end,Art community floor adult your single type. Pe...


# Load

## Read Config

In [7]:
with open('../../config/config.json') as fp:
    config = json.load(fp)
# config

##  Create engine

In [34]:
engine=create_engine(config['connection_strings']['dummy'])

# Insert data

In [51]:
    update_query = ""

In [52]:
for i in range(updated_products.shape[0]):
    update_query += "update products set name = '%s', description = '%s' where sku = '%s';"%(updated_products['name'][i], updated_products['description'][i], updated_products['sku'][i])

In [49]:
update_query

"update products set name = 'Bryce Jobes', set description = 'Art community floor adult your single type. Per back community former stock thing.' where sku = 'lay-raise-best-end';"

In [53]:
engine.execute(update_query)

In [54]:
foo = engine.execute("select * from products where sku='lay-raise-best-end'")

In [56]:
foo.all()

[('Bryce Jobes', 'lay-raise-best-end', 'Art community floor adult your single type. Per back community former stock thing.')]

In [57]:
foo = pd.DataFrame()

In [61]:
if foo is not None:
    print('bar')

bar
